In [1]:
# ! pip install --upgrade --quiet pymilvus langchain langchain-community langchainhub langchain-openai faiss-cpu unstructured
# ! docker-compose up -d

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PORTKEY_API_KEY = os.environ["PORTKEY_API_KEY"]

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI

from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders

import uuid

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
loader = DirectoryLoader("../city_data")
docs = loader.load_and_split(text_splitter=text_splitter)

In [4]:
embeddings = OpenAIEmbeddings()
db = Milvus.from_documents(
    docs, 
    embeddings,
    connection_args={"host": "127.0.0.1", "port": "19530"})

In [5]:
retriever = db.as_retriever()

In [6]:
tool = create_retriever_tool(
    retriever,
    "search_cities",
    "Searches and returns excerpts from Wikipedia entries of many cities.",
)
tools = [tool]

In [7]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [8]:
portkey_headers = createHeaders(
    api_key=PORTKEY_API_KEY,
    trace_id=uuid.uuid4(),
    provider="openai",
    config="pc-basic-b390c9"
)

In [9]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, base_url=PORTKEY_GATEWAY_URL, default_headers=portkey_headers, temperature=0)

In [10]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [14]:
result = agent_executor.invoke(
    {
        "input": "What is the size of San Francisco?"
    }
)

In [15]:
result["output"]

'The size of San Francisco is 46.9 square miles (121 square kilometers).'